In [ ]:
# Importação
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# ========================================
# 1. Geração de Dados Fictícios
# ========================================
np.random.seed(42)
n_samples = 1000

data_churn = {
    'uso_mensal_gb': np.random.exponential(20, n_samples),
    'meses_contrato': np.random.randint(1, 36, n_samples),
    'reclamacoes': np.random.poisson(1, n_samples),
    'suporte_tecnico': np.random.choice([0, 1], n_samples, p=[0.7, 0.3]),
    'churn': np.zeros(n_samples, dtype=int)
}

df_churn = pd.DataFrame(data_churn)

# Simular churn com lógica realista
prob_churn = (
    0.1 +
    0.3 * (df_churn['uso_mensal_gb'] > 50) +
    0.4 * (df_churn['meses_contrato'] < 12) +
    0.3 * (df_churn['reclamacoes'] > 2) +
    0.2 * df_churn['suporte_tecnico']
)

df_churn['churn'] = (np.random.random(n_samples) < prob_churn).astype(int)

# ========================================
# 2. ETL e Limpeza
# ========================================
print("Dataset Churn:")
print(df_churn.head())
print("\nDistribuição de Churn:")
print(df_churn['churn'].value_counts(normalize=True))

# ========================================
# 3. Visualizações
# ========================================
plt.figure(figsize=(14, 10))

# Churn por meses de contrato
plt.subplot(2, 2, 1)
sns.boxplot(x='churn', y='meses_contrato', data=df_churn)
plt.title('Meses de Contrato por Churn')
plt.xlabel('Churn (1=Sim)')

# Uso mensal por churn
plt.subplot(2, 2, 2)
sns.boxplot(x='churn', y='uso_mensal_gb', data=df_churn)
plt.title('Uso Mensal (GB) por Churn')

# Reclamações
plt.subplot(2, 2, 3)
sns.countplot(x='reclamacoes', hue='churn', data=df_churn)
plt.title('Reclamações vs Churn')

# Suporte técnico
plt.subplot(2, 2, 4)
sns.countplot(x='suporte_tecnico', hue='churn', data=df_churn)
plt.title('Suporte Técnico vs Churn')
plt.xlabel('Solicitou Suporte (1=Sim)')

plt.tight_layout()
plt.show()

# ========================================
# 4. Modelo: Regressão Logística
# ========================================
X = df_churn[['uso_mensal_gb', 'meses_contrato', 'reclamacoes', 'suporte_tecnico']]
y = df_churn['churn']

# Padronização
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

model_log = LogisticRegression(random_state=42)
model_log.fit(X_train, y_train)

y_pred = model_log.predict(X_test)
y_prob = model_log.predict_proba(X_test)[:, 1]

# ========================================
# 5. Avaliação
# ========================================
print("\n" + "="*50)
print("REGRESSÃO LOGÍSTICA - RESULTADOS")
print("="*50)
print(classification_report(y_test, y_pred))

# Matriz de confusão
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Não Churn', 'Churn'], yticklabels=['Não Churn', 'Churn'])
plt.title('Matriz de Confusão')
plt.ylabel('Real')
plt.xlabel('Previsto')
plt.show()

# AUC-ROC
auc = roc_auc_score(y_test, y_prob)
print(f"AUC-ROC: {auc:.4f}")

# Curva ROC
fpr, tpr, _ = roc_curve(y_test, y_prob)
plt.figure(figsize=(7, 6))
plt.plot(fpr, tpr, label=f'ROC (AUC = {auc:.3f})', color='darkorange')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('Taxa de Falsos Positivos')
plt.ylabel('Taxa de Verdadeiros Positivos')
plt.title('Curva ROC - Regressão Logística')
plt.legend()
plt.show()

# Coeficientes
print("\nCoeficientes do modelo (odds):")
for feature, coef in zip(X.columns, model_log.coef_[0]):
    print(f"  {feature}: {np.exp(coef):.3f}x mais chance de churn")